In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from netam import framework, models
from netam.framework import calculate_loss
from epam.torch_common import pick_device

In [2]:
shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
all_df = pd.read_csv(shmoof_data_path)

# Here's the fraction of sequences of length more than 410
(all_df["parent"].str.len() > 410).sum() / len(all_df)

0.00403216242498992

In [3]:
train_df, val_df = framework.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv") #, sample_count=5000)


In [4]:
kmer_length = 3
max_length = 410

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 35830 training examples and 13186 validation examples


In [5]:
class CNNModel(nn.Module):
    def __init__(self, dataset, embedding_dim, num_filters, kernel_size):
        super(CNNModel, self).__init__()
        self.kmer_count = len(dataset.kmer_to_index)

        self.kmer_embedding = nn.Embedding(self.kmer_count, embedding_dim)

        # Convolutional layer
        self.conv = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_size, padding='same')

        self.linear = nn.Linear(in_features=num_filters, out_features=1)

    def forward(self, encoded_parents, masks):
        kmer_embeds = self.kmer_embedding(encoded_parents)
        # Need to do transpose because conv1D expects the channel dimension
        # (embedding_dim) to be before the along-sequence dimension. 
        kmer_embeds = kmer_embeds.permute(0, 2, 1)

        # Apply convolutional layer
        conv_out = F.relu(self.conv(kmer_embeds))
        conv_out = conv_out.permute(0, 2, 1)

        # Apply linear layer
        log_rates = self.linear(conv_out).squeeze(-1)

        # Exponentiate to get rates
        rates = torch.exp(log_rates * masks)

        return rates


model = CNNModel(train_dataset, embedding_dim=12, num_filters=13, kernel_size=7)

model.to(device)


CNNModel(
  (kmer_embedding): Embedding(65, 12)
  (conv): Conv1d(12, 13, kernel_size=(7,), stride=(1,), padding=same)
  (linear): Linear(in_features=13, out_features=1, bias=True)
)

In [7]:
burrito = framework.Burrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-5, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=50)

starting training...
Epoch [0/50]	 Loss: 0.058427606	 Val Loss: 0.066611466
Epoch [1/50]	 Loss: 0.058895127	 Val Loss: 0.066749167
Epoch [2/50]	 Loss: 0.058507647	 Val Loss: 0.066771644
Epoch [3/50]	 Loss: 0.058480443	 Val Loss: 0.066700835
Epoch [4/50]	 Loss: 0.058442706	 Val Loss: 0.066774304
Epoch [5/50]	 Loss: 0.058428341	 Val Loss: 0.066680854
Epoch [6/50]	 Loss: 0.058429926	 Val Loss: 0.066661189
Epoch [7/50]	 Loss: 0.058385966	 Val Loss: 0.066658454
Epoch [8/50]	 Loss: 0.058428884	 Val Loss: 0.066707124
Epoch [9/50]	 Loss: 0.05840453	 Val Loss: 0.066634798
Epoch [10/50]	 Loss: 0.058391592	 Val Loss: 0.066730997
Epoch [11/50]	 Loss: 0.058412626	 Val Loss: 0.066734078
Epoch [12/50]	 Loss: 0.058366708	 Val Loss: 0.066653705
Epoch [13/50]	 Loss: 0.058376967	 Val Loss: 0.066677899
Epoch [14/50]	 Loss: 0.058376679	 Val Loss: 0.066660937
Epoch 00014: reducing learning rate of group 0 to 2.0000e-02.
Epoch [15/50]	 Loss: 0.058126266	 Val Loss: 0.06644326
Epoch [16/50]	 Loss: 0.058001318	

In [16]:
len(val_dataset.all_kmers), len(our_val_dataset.all_kmers)

(65, 65)

In [19]:
nicknames = ["51", "small", "13"]
val_losses = []
for nickname in nicknames:
    our_train_df, our_val_df = framework.load_shmoof_dataframes(shmoof_data_path, val_nickname=nickname)
    our_val_dataset = framework.SHMoofDataset(our_val_df, kmer_length=kmer_length, max_length=max_length)
    our_val_dataset.to(device)
    our_val_loader = torch.utils.data.DataLoader(our_val_dataset, batch_size=1024, shuffle=False)

    print(f"evaluating on {nickname}...")
    # our_burrito = framework.Burrito(our_train_dataset, our_val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-4, l2_regularization_coeff=1e-6)
    val_losses.append(burrito.process_data_loader(our_val_loader, train_mode=False))

val_df = pd.DataFrame({"nickname": nicknames, "loss": val_losses})
val_df
    

evaluating on 51...
evaluating on small...
evaluating on 13...


,nickname,loss
0,51,0.059682
1,small,0.054704
2,13,0.066406


In [ ]:
import optuna
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

batch_size = 1024
learning_rate = 0.1

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


def objective(trial):
    # Define the hyperparameters to optimize
    embedding_dim = trial.suggest_int("embedding_dim", 5, 20)
    num_filters = trial.suggest_int("num_filters", 5, 20)
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5, 7])

    # Initialize the model with the suggested hyperparameters
    model = CNNModel(train_dataset, embedding_dim=embedding_dim, num_filters=num_filters, kernel_size=kernel_size)
    model.to(device)
    
    # Define optimizer, loss criterion, etc.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # You might need to adjust this depending on your train and validation loop implementation
    epochs = 40  # Number of epochs to train the model
    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        for encoded_parents, masks, mutation_indicators in train_loader:
            optimizer.zero_grad()
            rates = model(encoded_parents, masks)
            loss = calculate_loss(rates, masks, mutation_indicators)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for encoded_parents, masks, mutation_indicators in val_loader:
                rates = model(encoded_parents, masks)
                loss = calculate_loss(rates, masks, mutation_indicators)
                val_loss += loss.item() * encoded_parents.size(0)
        val_loss /= len(val_loader.dataset)

        if val_loss < best_val_loss:
            best_val_loss = val_loss

    return best_val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print the results
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-18 13:01:45,504] A new study created in memory with name: no-name-01189937-bf16-4618-8e1a-7c37a9505fa0
[I 2023-11-18 13:05:16,096] Trial 0 finished with value: 0.06628789302387134 and parameters: {'embedding_dim': 10, 'num_filters': 17, 'kernel_size': 7}. Best is trial 0 with value: 0.06628789302387134.
[I 2023-11-18 13:08:34,205] Trial 1 finished with value: 0.06644169138918223 and parameters: {'embedding_dim': 8, 'num_filters': 6, 'kernel_size': 5}. Best is trial 0 with value: 0.06628789302387134.
[I 2023-11-18 13:12:28,156] Trial 2 finished with value: 0.06641458698723397 and parameters: {'embedding_dim': 15, 'num_filters': 19, 'kernel_size': 5}. Best is trial 0 with value: 0.06628789302387134.
[I 2023-11-18 

KeyboardInterrupt: 